In [ ]:
# general
import json
import collections
import math
import uuid
import random
import pickle
import os
import time
from itertools import islice
import numpy as np
from tqdm.notebook import tqdm
import open3d as o3d

from src.geometry import sq_distance, vector_mag, sq_dist_vect
from src.visualisation import *
from src.elements import *
from src.chamfer import *

In [ ]:
blueprint = 'data/sample.ifc'
data_path = "/mnt/c/data/3D_CAD/"
dist_threshold = 0.01 # distance threshold for pipe proximity
centerline_angle_threshold = math.radians(10) # angle threshold for two pipes to be considered parallel
centerline_dist_threshold = 0.0005 # centerline distance threshold pipes to be to considered connected

In [ ]:
# load data
with open(data_path + 'edges_westdeckbox.pkl', 'rb') as f:
    edges = pickle.load(f)
with open(data_path + 'nodes_westdeckbox.pkl', 'rb') as f:
    node_info = pickle.load(f)
    nodes = node_info[0]
    


In [ ]:
print(nodes[0], edges[0])

In [ ]:
e1 = [e[0] for e in edges]
e2 = [e[1] for e in edges]
print(max(e1), max(e2))
print(len(nodes))
print(len(edges))

In [ ]:
print(type(nodes[0][3]))

In [ ]:
# find closest pair of edges satisfying a minimum distance criteria
def edge_proximity_criteria(e1, e2, threshold):
    nearby_pair = None
    min_dist = math.inf
    d = sq_dist_vect(e1[0], e2[0])
    if (d < threshold):
        nearby_pair = [0, 0]
        min_dist = d
    d = sq_dist_vect(e1[1], e2[0])
    if (d < threshold and d < min_dist): 
        nearby_pair = [1, 0]
        min_dist = d
    d = sq_dist_vect(e1[0], e2[1])
    if (d < threshold and d < min_dist): 
        nearby_pair = [0, 1]
        min_dist = d    
    d = sq_dist_vect(e1[1], e2[1]) 
    if (d < threshold and d < min_dist): 
        nearby_pair = [1, 1]
        min_dist = d
    return nearby_pair


In [ ]:
# get pipe neighbours

# find edges of pipes
# format - category, centerpoint, bounding box, principal direction, element id
pipes = [[n[0], np.array(n[1]), np.array(n[2]), np.array(n[3]), n[4]] for n in nodes if n[0]==3]
pipe_edges = [(n[1] - (max(n[2])/2 * n[3]),
               n[1] + (max(n[2])/2 * n[3])) for n in pipes]
# print(len(pipe_edges))
# print(pipe_edges[0])

# compare edges to find close-by edges
nearby_edges = []
for i, pe1 in tqdm(enumerate(pipe_edges)):
    t1 = time.perf_counter()
    for j, pe2 in enumerate(pipe_edges[i+1:]):
        nearby_pair = edge_proximity_criteria(pe1, pe2, dist_threshold)
        if nearby_pair is not None:
            nearby_edges.append(((i,j+i+1), nearby_pair))
    t2 = time.perf_counter()
    if (t2 - t2) > 0.1:
        print((t2 - t1), nearby_pair, min_dist)            
print(len(nearby_edges), nearby_edges[0])

In [ ]:
print(pipe_edges[0])
print(pipes[0])
lens = [max(p[2]) for p in pipes]
print(max(lens), min(lens), sum(lens)/len(lens))

In [ ]:
def edge_match(a, b):
    e = None
    if (a[0][0] == b[0][0] and a[1][0] == b[1][0]):
        e = 1
    elif (a[0][1] == b[0][0] and a[1][1] == b[1][0]):
        e = 0
    elif (a[0][0] == b[0][1] and a[1][0] == b[1][1]):
        e = 1
    elif (a[0][1] == b[0][1] and a[1][1] == b[1][1]):
        e = 0
    
    if e is not None:
        return ((a[0][e], b[0][0], b[0][1]), (a[1][e], b[1][0], b[1][1]))
    else:
        return None

In [ ]:
# rough check for tees
possible_tees = []
for i, ne1 in tqdm(enumerate(nearby_edges)):
    for j, ne2 in enumerate(nearby_edges[i+1:]):
        pt = edge_match(ne1, ne2)
        if pt is not None:
            possible_tees.append(pt)
            
print(len(possible_tees), possible_tees[0])

In [ ]:
def get_radius_from_bbox(bbox):
    sides = bbox[bbox.argsort()[:2]]
    return (sum(sides)/4)


In [ ]:
def radius_check(pipe_list, thresh):
    radii = [get_radius_from_bbox(p[2]) for p in pipe_list]
    check = True
    
    for r1 in radii:
        if not check:
            break
        for r2 in radii:
            if (r1/r2 < thresh) or (r2/r1 < thresh):
                check = False
                break
    return check


def get_centerline_deviation(ad, bd):
    centerline_deviation = np.arccos( np.dot(ad, bd))
    if centerline_deviation > np.pi/2:
        centerline_deviation = np.pi - centerline_deviation
    return centerline_deviation


def get_centerline_distance(a, b):
    centerline_connecting_line = np.cross(a[3], b[3])
    edge_connecting_line = b[1] - a[1]
    centerline_distance = (abs(np.dot(centerline_connecting_line, 
                                          edge_connecting_line)) / 
                           vector_mag(centerline_connecting_line))
    return centerline_distance


def pipe_check(a, b):
    # radius check
    if radius_check([a,b], 0.8):
        
        # centerline direction check -
        centerline_deviation = get_centerline_deviation(a[3], b[3])
        if centerline_deviation < centerline_angle_threshold:

            # centerline proximity check        
            centerline_distance = get_centerline_distance(a, b)
            if centerline_distance < centerline_dist_threshold:
                return True
    return False


def elbow_check(a, b, thresh = 0.4):
    # radius check
    if radius_check([a,b], thresh):
        
        # centerline direction check -
        centerline_deviation = get_centerline_deviation(a[3], b[3])
        if centerline_deviation > centerline_angle_threshold:

            # centerline proximity check        
            centerline_distance = get_centerline_distance(a, b)
            if centerline_distance < centerline_dist_threshold:
                return True
    return False
        

In [ ]:
# check for tees
# if two of the elements seem to be connected in a straight line and the other is placed like an elbow, its considered a tee
tee_connections = []
for i, pt in tqdm(enumerate(possible_tees)):
   
    # check for straight line connection
    pipe_connection = False
    if pipe_check(pipes[pt[0][0]], pipes[pt[0][1]]):
        if elbow_check(pipes[pt[0][2]], pipes[pt[0][0]]) and elbow_check(pipes[pt[0][2]], pipes[pt[0][0]]):
            pipe_connection = True
            pipe_pair = [0,1]
            other = 2
    if pipe_check(pipes[pt[0][0]], pipes[pt[0][2]]):
        if elbow_check(pipes[pt[0][1]], pipes[pt[0][0]]) and elbow_check(pipes[pt[0][1]], pipes[pt[0][2]]):       
            pipe_connection = True
            pipe_pair = [0,2]
            other = 1
    if pipe_check(pipes[pt[0][1]], pipes[pt[0][2]]):
        if elbow_check(pipes[pt[0][0]], pipes[pt[0][1]]) and elbow_check(pipes[pt[0][0]], pipes[pt[0][2]]):
            pipe_connection = True
            pipe_pair = [1,2]
            other = 0
        
    # check for angled connection
    if pipe_connection:
        if elbow_check(pipes[pt[0][pipe_pair[0]]], pipes[pt[0][other]]):
            tee_connections.append((pt, pipe_pair, other))
            
print(len(tee_connections), tee_connections[0])
        

In [ ]:
# # check for elbows or connected pipes
# pipe_connections = []
# elbow_connections = []
# for i, ne in enumerate(nearby_edges): 

#     if pipe_check(ne[0], ne[1]):
#         pipe_connections.append(ne)
#         continue
    
#     if elbow_check(ne[0], ne[1]):
#         elbow_connections.append(ne)
            
# # remove detected tees

In [ ]:
# check for elbows


In [ ]:
def generate_tee_cloud(preds, refine=True):
    # read params
    r1, l1, r2, l2 = preds[0], preds[1], preds[2], preds[3]
    d1 = get_direction_from_trig(preds, 7)
    d2 = get_direction_from_trig(preds, 13)
    p2 = [preds[4], preds[5], preds[6]]
    p1 = [p2[i] - ((l1*d1[i])/2) for i in range(3)]
    
    # get new coordinate frame of tee
    old_z = (0., 0., 1.)
    x_axis = vector_normalise(np.cross(d1, old_z))
    y_axis = vector_normalise(np.cross(d1, x_axis))
    
    # sample points on main tube
    no_of_axis_points = 50    
    no_of_ring_points = 40
    tube1_points = get_cylinder_points(no_of_axis_points, no_of_ring_points, 
                                      r1, l1, p1, d1, x_axis, y_axis)

    # sample points on secondary tube
    x_axis = vector_normalise(np.cross(d2, old_z))
    y_axis = vector_normalise(np.cross(d2, x_axis))   
    tube2_points = get_cylinder_points(no_of_axis_points, no_of_ring_points, 
                                      r2, l2, p2, d2, x_axis, y_axis)
    
    if not refine:
        return tube1_points + tube2_points
    else:
    
        # remove points from secondary tube in main tube
        tube2_points = np.array(tube2_points)
        p1, p2 = np.array(p1), np.array(p2)
        p2p1 = p2-p1
        p2p1_mag = vector_mag(p2p1)
        tube2_points_ref = []
        
        for q in tube2_points:
            dist = vector_mag(np.cross((q-p1), (p2p1))) / p2p1_mag
            #print(dist)
            if dist > r1:
                tube2_points_ref.append(q.tolist())
                
        # remove points from main tube in secondary tube
        tube1_points = np.array(tube1_points)
        p3 = np.array(p2 + d2)
        p2p3 = p2-p3
        p2p3_mag = vector_mag(p2p3)
        tube1_points_ref = []
        
        for q in tube1_points:
            dist = vector_mag(np.cross((q-p3), (p2p3))) / p2p3_mag
            cos_theta = np.dot(q-p2, p2p3)
            if dist > r2 or cos_theta > 0:
                tube1_points_ref.append(q.tolist())
        
        # make sure not all points are deleted if predictions are very wrong
        thresh = 50
        if len(tube1_points_ref) < thresh and len(tube2_points_ref) < thresh:
            return (tube1_points.tolist() + tube2_points.tolist())
        elif len(tube2_points_ref) < thresh:
            return (tube1_points_ref + tube2_points.tolist())
        elif len(tube1_points_ref) < thresh:
            return (tube1_points.tolist() + tube2_points_ref)
        else:
            return (tube1_points_ref + tube2_points_ref)

In [ ]:
# checking and visualisation

elbows = [[n[0], np.array(n[1]), np.array(n[2]), np.array(n[3]), n[4]] for n in nodes if n[0]==1]
tees = [[n[0], np.array(n[1]), np.array(n[2]), np.array(n[3]), n[4]] for n in nodes if n[0]==2]

print(len(tees))

def visualise_pipes(pipes, return_type="cloud"):
    clouds = []
    preds = []
    for pipe in tqdm(pipes):
        r = get_radius_from_bbox(pipe[2])
        l = max(pipe[2])
        d = pipe[3].tolist()
        p = pipe[1].tolist()
        params = [r, l] + p

        for i in range(3):
            params.append(math.sin(d[i]))
            params.append(math.cos(d[i]))
        
        preds.append(params)
        if return_type == "cloud":
            cld = np.array(generate_pipe_cloud(params, scale = True))
            clouds.append(cld)
            
    ne = [len(cl) for cl in clouds if len(cl)>0]

    print("sizes", len(clouds), len(pipes), len(ne))

    if return_type == "cloud":
        return np.concatenate(clouds)
    else:
        return preds
    

def visualise_tees(tee_connections, blueprint, return_type="cloud"):
    # setup ifc
    ifc = setup_ifc_file(blueprint)
    owner_history = ifc.by_type("IfcOwnerHistory")[0]
    project = ifc.by_type("IfcProject")[0]
    context = ifc.by_type("IfcGeometricRepresentationContext")[0]
    floor = ifc.by_type("IfcBuildingStorey")[0]
    scale = 1
    clouds = []
    preds = []
    skip_count = 0

    ifc_info = {"owner_history": owner_history,
        "project": project,
       "context": context, 
       "floor": floor}
    
    # calculate tee parameters
    for tee in tqdm(tee_connections):
        pipe_pair_ids = [tee[0][0][tee[1][0]], tee[0][0][tee[1][1]]]
        pipe_pair = (pipes[pipe_pair_ids[0]], pipes[pipe_pair_ids[1]])
        other_id = tee[0][0][tee[2]]
        other = pipes[other_id]
        pipe_edge_ids = [tee[0][1][tee[1][0]], tee[0][1][tee[1][1]]]
        other_edge_id = tee[0][1][tee[2]]
        
        r1 = (get_radius_from_bbox(pipe_pair[0][2]) + get_radius_from_bbox(pipe_pair[1][2]))/2
        r2 = get_radius_from_bbox(other[2])
       # print('r', r1, r2)     

        l1_edges = [pipe_edges[pipe_pair_ids[0]][pipe_edge_ids[0]], 
                    pipe_edges[pipe_pair_ids[1]][pipe_edge_ids[1]]]
        l1 = np.sqrt(sq_dist_vect(l1_edges[0], l1_edges[1]))
        l2_edges = [(l1_edges[0] + l1_edges[1])/2, pipe_edges[other_id][other_edge_id]]
        l2 = np.sqrt(sq_dist_vect(l2_edges[0], l2_edges[1]))
        #print('l', l1, l2)
        
        r1, r2, l1, l2 = r1*scale, r2*scale, l1*scale, l2*scale

        p1 = (l1_edges[0]*scale).tolist()
        p2 = (l2_edges[0]*scale).tolist()
        d1 = vector_normalise(l1_edges[1] - l1_edges[0])
        
        # additional check by calculating d1 again
        d1_alternative = pipe_pair[0][3].tolist()
        d1_deviation =  get_centerline_deviation(d1, d1_alternative)
        if d1_deviation > centerline_angle_threshold:
            skip_count += 1
            continue
        
        d2 = vector_normalise(l2_edges[1] - l2_edges[0])
        #print('pd', p1, p2, d1, d2)
        
        # temporary: format to predictions format to generate visualisation
        params = [r1, l1, r2, l2] + p2
        
        for d in [d1, d2]:
            for i in range(3):
                params.append(math.sin(d[i]))
                params.append(math.cos(d[i]))
        
        #print(len(params), params)
        if return_type == "cloud":
            cld = np.array(generate_tee_cloud(params))
            clouds.append(cld)
        else:
            preds.append(params)
            #create_IfcTee(r1, r2, l1, l2, d1, d2, p1, p2, ifc, ifc_info)

    print(len(tee_connections) - skip_count)
    if return_type == "cloud":
        return np.concatenate(clouds)
    else:
        return preds



In [ ]:
tee_points = visualise_tees(tee_connections, blueprint)

In [ ]:
print(tee_points.shape)

In [ ]:
tee_points = o3d.utility.Vector3dVector(tee_points)
tee_cloud = o3d.geometry.PointCloud()
tee_cloud.points = tee_points
o3d.io.write_point_cloud("bp_tee_centerline.pcd", tee_cloud)


In [ ]:
pipe_points = visualise_pipes(pipes)


In [ ]:
pipe_points = o3d.utility.Vector3dVector(pipe_points)
pipe_cloud = o3d.geometry.PointCloud()
pipe_cloud.points = pipe_points
o3d.io.write_point_cloud("bp_pipe_centerline.pcd", pipe_cloud)


In [ ]:
# visualise ifc and predictions together
# tee_ifc = ifcopenshell.open(data_path + 'deckboxtee.ifc')

# viewer = vis_ifc_and_cloud(tee_ifc, [tee_points])

In [ ]:
viewer

In [ ]:
ifc.write("bp_tee_vis.ifc")